# Assignment #2: Language models
Author: Pierre Nugues

## Objectives

The objectives of this assignment are to:
* Write a program to find n-gram statistics
* Compute the probability of a sentence
* Know what a language model is
* Write a short report of 1 to 2 pages on the assignment

## Submission

Once you have written all the missing code and run all the cells, you will submit your notebook to an automatic marking system. Do not erase the content of the cells as we will possibly check your programs manually.
The submission instructions are at the bottom of the notebook.

## Organization

* Each group will have to write Python programs to count unigrams, bigrams, and trigrams in a corpus of approximately one million words and to determine the probability of a sentence.
* You can test you regular expression using the regex101.com site
* Each student will have to write a short report of one to two pages and comment briefly the results. In your report, you must produce the tabulated results of your analysis as described below.

## Programming

### Imports

Some imports you may need. Add others as needed.

In [48]:
import bz2
import math
import os
import regex as re
import requests
import sys
from zipfile import ZipFile

### Collecting and analyzing a corpus

Retrieve a corpus of novels by Selma Lagerl&ouml;f from this URL:
<a href="https://github.com/pnugues/ilppp/blob/master/programs/corpus/Selma.txt">
    <tt>https://github.com/pnugues/ilppp/blob/master/programs/corpus/Selma.txt</tt>
</a>. The text of these novels was extracted
from <a href="https://litteraturbanken.se/forfattare/LagerlofS/titlar">Lagerlöf arkivet</a> at
<a href="https://litteraturbanken.se/">Litteraturbanken</a>.

In [49]:
# You may have to adjust the path
corpus = open('Selma.txt', encoding='utf8').read()

Run the <a href="https://github.com/pnugues/ilppp/tree/master/programs/ch02/python">concordance
program </a> to print the lines containing a specific word, for instance <i>Nils</i>.

In [50]:
pattern = 'Nils Holgersson'
width = 25

In [51]:
# spaces match tabs and newlines
pattern = re.sub(' ', '\\s+', pattern)
# Replaces newlines with spaces in the text
clean_corpus = re.sub('\s+', ' ', corpus)
concordance = ('(.{{0,{width}}}{pattern}.{{0,{width}}})'
               .format(pattern=pattern, width=width))
for match in re.finditer(concordance, clean_corpus):
    print(match.group(1))
# print the string with 0..width characters on either side

Selma Lagerlöf Nils Holgerssons underbara resa genom Sv
! Se på Tummetott! Se på Nils Holgersson Tummetott!» Genast vände
r,» sade han. »Jag heter Nils Holgersson och är son till en husma
lden. »Inte är det värt, Nils Holgersson, att du är ängslig eller
 i dem. På den tiden, då Nils Holgersson drog omkring med vildgäs
ulle allt visa honom vad Nils Holgersson från Västra Vemmenhög va
om ägde rum det året, då Nils Holgersson for omkring med vildgäss
m vad det kan kosta dem. Nils Holgersson hade inte haft förstånd 
de det inte mer sägas om Nils Holgersson, att han inte tyckte om 
 Rosenbom?» För där stod Nils Holgersson mitt uppe på Rosenboms n
 Med ens fingo de syn på Nils Holgersson, och då sköt den store v
vila. När vildgässen och Nils Holgersson äntligen hade letat sig 
 slags arbetare. Men vad Nils Holgersson inte såg, det var, att s
nde han fråga, och om då Nils Holgersson sade nej, började han ge
de lille Mats, och om nu Nils Holgersson också hade tegat, så had
åg så försmädlig ut,

Run a simple <a href="https://github.com/pnugues/ilppp/tree/master/programs/ch05/python">tokenization
program</a> on your corpus.

In [52]:
def tokenize(text):
    words = re.findall('\p{L}+', text)
    return words

In [53]:
words = tokenize(corpus)
words[:10]

['Selma',
 'Lagerlöf',
 'Nils',
 'Holgerssons',
 'underbara',
 'resa',
 'genom',
 'Sverige',
 'Första',
 'bandet']

In [54]:
def unigrams(words):
    frequency = {}
    for i in range(len(words)):
        if words[i] in frequency:
            frequency[words[i]] += 1
        else:
            frequency[words[i]] = 1
    return frequency


In [55]:
wc = unigrams(tokenize(corpus.lower()))

In [56]:
wc["den"]

11624

Count the number of unique words in the original corpus and when setting all the words in lowercase

Original text

In [57]:
def count():
    wordlist = tokenize(corpus)
    return len(set(wordlist))
count()
#44256

44256

Lowercased text

In [58]:
def count_lower():
    lower_word_list = tokenize(corpus.lower())
    return len(set(lower_word_list))
count_lower()
#41032

41032

### Segmenting a corpus

You will write a program to tokenize your text, insert `<s>` and `</s>` tags to delimit sentences, and set all the words in lowercase letters. In the end, you will only keep the words.

#### Normalizing 

Write a regular expression that matches all the characters that are neither a letter nor a punctuation sign. The punctuations signs will be the followings: `.;:?!`. In your regex, use the same order. For the definition of a letter, use a Unicode regex. You will call the regex string `nonletter`

In [59]:
#regex = re.compile("[^\.+ | \!+ | \?+ | \:+ | \;+ | \p{L}+]")
regex = re.compile("[^\P{P}\.\?!]")
nonletter = regex

Write a `clean()` function that replaces all the characters that are neither a letter nor a punctuation sign with a space. The punctuations signs will be the followings: `.;:?!`.   For the sentence:

_En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa._

the result will be:

`En gång hade de på Mårbacka en barnpiga som hette Back Kajsa.`

In [60]:
def clean(words):
    tmp = re.sub(nonletter, " ", words)
    return re.sub(r"\s\s+", " ", tmp) # Removes double space

In [61]:
test_para = 'En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa. \
Hon var nog sina tre alnar lång, hon hade ett stort, grovt ansikte med stränga, mörka drag, \
hennes händer voro hårda och fulla av sprickor, som barnens hår fastnade i, \
när hon kammade dem, och till humöret var hon dyster och sorgbunden.'

In [62]:
tmp = clean(test_para)
#re.findall(nonletter, test_para)
tmp

'En gång hade de på Mårbacka en barnpiga som hette Back Kajsa. Hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden.'

#### Segmenter

In this section, you will write a sentence segmenter that will delimit each sentence with `</s>` and `<s>` symbols. For example the sentence:

_En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa._

will be bracketed as:

`<s> En gång hade de på Mårbacka en barnpiga som hette Back-Kajsa </s>`

As algorithm, you will use a simple heuristics to detect the sentence boundaries: A sentence starts with a capital letter and ends with a period-equivalent punctuation sign. You will write a regex to match these boundaries with a regular expression and you will insert `</s>\n<s>` symbols with a substitution function.

##### Detecting sentence boundaries

Write a regular expression that matches a punctuation, a sequence of spaces, and an uppercase letter. Call this regex string `sentence_boundaries`. In the regex, you will remember the value of the uppercase letter using a backreference. Use the Unicode regexes for the letters and the spaces.

In [63]:
ex1 = re.findall(r"(\p{Lu}).+?([.?!;:])", test_para)

ex2 = re.findall(r"((\p{Lu}).+?([.?!;:]))", test_para)

print(ex1)
print(ex2)

sentence_boundries = r"((\p{Lu}).+?)[.?!;:]"
re.findall(sentence_boundries, test_para)

[('E', '.'), ('H', '.')]
[('En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa.', 'E', '.'), ('Hon var nog sina tre alnar lång, hon hade ett stort, grovt ansikte med stränga, mörka drag, hennes händer voro hårda och fulla av sprickor, som barnens hår fastnade i, när hon kammade dem, och till humöret var hon dyster och sorgbunden.', 'H', '.')]


[('En gång hade de på Mårbacka en barnpiga, som hette Back-Kajsa', 'E'),
 ('Hon var nog sina tre alnar lång, hon hade ett stort, grovt ansikte med stränga, mörka drag, hennes händer voro hårda och fulla av sprickor, som barnens hår fastnade i, när hon kammade dem, och till humöret var hon dyster och sorgbunden',
  'H')]

##### Replacement markup

Write a string to replace the matched boundaries with the sentence boundary markup. Remember that a sentence ends with `</s>` and starts with `<s>` and that there is one sentence per line. Hint: The markup is `</s>\n<s>`. Remember also that the first letter of your sentence is in a regex backreference. Call the regex string `sentence_markup`.

In [64]:
sentence_markup = r"\1 </s>\n<s>"
print(sentence_markup)

\1 </s>\n<s>


##### Applying the substitution

Use your regexes to segment your text. Use the string `sentence_boundaries`, `sentence_markup`, and `test_para` as input and `text` as output.

In [65]:
# Write your code here
text = re.sub(sentence_boundries, sentence_markup, clean(test_para))
print(text)

En gång hade de på Mårbacka en barnpiga som hette Back Kajsa </s>
<s> Hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden </s>
<s>


In [66]:
print(text)

En gång hade de på Mårbacka en barnpiga som hette Back Kajsa </s>
<s> Hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden </s>
<s>


Insert markup codes in the beginning and end of the text

In [67]:
# Write your code here
new_markup = r"<s> \1 </s>\n"
text = re.sub(sentence_boundries, new_markup, clean(test_para))
text = re.sub(" <s>","<s>", text)
print(text)


<s> En gång hade de på Mårbacka en barnpiga som hette Back Kajsa </s>
<s> Hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden </s>



In [68]:
print(text)

<s> En gång hade de på Mårbacka en barnpiga som hette Back Kajsa </s>
<s> Hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden </s>



Replace the space duplicates with one space and remove the punctuation signs. For the spaces, use the Unicode regex.

In [69]:
new_markup = r"<s> \1 </s>\n"
text = re.sub(sentence_boundries, new_markup, clean(test_para))
text = re.sub(r"[^\S\r\n]+", " ", text) #Remove the spaces but not the newlines
print(text)

<s> En gång hade de på Mårbacka en barnpiga som hette Back Kajsa </s>
 <s> Hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden </s>



In [70]:
print(text)

<s> En gång hade de på Mårbacka en barnpiga som hette Back Kajsa </s>
 <s> Hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden </s>



Write a `segment_sentences(text)` function to gather the code in the Segmenter section and set the text in lowercase

In [71]:
# Write your code here
def segment_sentences(text):
    markup = r"<s> \1 </s> \n"
    tmp = re.sub(sentence_boundries, markup, clean(text))
    tmp = re.sub(r"\s\s+", " ", tmp) #Removes double spaces and newlines
    tmp = tmp.lower()
    return tmp
print(segment_sentences(test_para))

<s> en gång hade de på mårbacka en barnpiga som hette back kajsa </s> <s> hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden </s> 


In [72]:
print(segment_sentences(test_para))

<s> en gång hade de på mårbacka en barnpiga som hette back kajsa </s> <s> hon var nog sina tre alnar lång hon hade ett stort grovt ansikte med stränga mörka drag hennes händer voro hårda och fulla av sprickor som barnens hår fastnade i när hon kammade dem och till humöret var hon dyster och sorgbunden </s> 


Estimate roughly the accuracy of your program.

#### Tokenizing the corpus

Clean and segment the corpus

In [73]:
# Write your code here
segmented_corpus = segment_sentences(corpus)
segmented_corpus[-557:]

's> hon hade fått större kärlek av sina föräldrar än någon annan han visste och sådan kärlek måste vändas i välsignelse </s> <s> då prästen sade detta kom alla människor att se bort mot klara gulla och de förundrade sig över vad de såg </s> <s> prästens ord tycktes redan ha gått i uppfyllelse </s> <s> där stod klara fina gulleborg ifrån skrolycka hon som var uppkallad efter själva solen vid sina föräldrars grav och lyste som en förklarad </s> <s> hon var likaså vacker som den söndagen då hon gick till kyrkan i den röda klänningen om inte vackrare </s> '

The result should be a normalized text without punctuation signs where all the sentences are delimited with `<s>` and `</s>` tags. The five last lines of the text should look like this.

In [74]:
print(corpus[-557:])

 ord till Klara Gulla. Hon hade fått större kärlek av sina föräldrar än någon annan han visste, och sådan kärlek måste vändas i välsignelse.

Då prästen sade detta, kom alla människor att se bort mot Klara Gulla, och de förundrade sig över vad de såg.

Prästens ord tycktes redan ha gått i uppfyllelse. Där stod Klara Fina Gulleborg ifrån Skrolycka, hon, som var uppkallad efter själva solen, vid sina föräldrars grav och lyste som en förklarad.

Hon var likaså vacker som den söndagen, då hon gick till kyrkan i den röda klänningen, om inte vackrare.








You will now create a list of words from your string. You will consider that a space or a carriage return is an item separator

In [75]:
separated = segmented_corpus.strip().split(" ")
print(separated[-101:])

['<s>', 'hon', 'hade', 'fått', 'större', 'kärlek', 'av', 'sina', 'föräldrar', 'än', 'någon', 'annan', 'han', 'visste', 'och', 'sådan', 'kärlek', 'måste', 'vändas', 'i', 'välsignelse', '</s>', '<s>', 'då', 'prästen', 'sade', 'detta', 'kom', 'alla', 'människor', 'att', 'se', 'bort', 'mot', 'klara', 'gulla', 'och', 'de', 'förundrade', 'sig', 'över', 'vad', 'de', 'såg', '</s>', '<s>', 'prästens', 'ord', 'tycktes', 'redan', 'ha', 'gått', 'i', 'uppfyllelse', '</s>', '<s>', 'där', 'stod', 'klara', 'fina', 'gulleborg', 'ifrån', 'skrolycka', 'hon', 'som', 'var', 'uppkallad', 'efter', 'själva', 'solen', 'vid', 'sina', 'föräldrars', 'grav', 'och', 'lyste', 'som', 'en', 'förklarad', '</s>', '<s>', 'hon', 'var', 'likaså', 'vacker', 'som', 'den', 'söndagen', 'då', 'hon', 'gick', 'till', 'kyrkan', 'i', 'den', 'röda', 'klänningen', 'om', 'inte', 'vackrare', '</s>']


The five last lines of the corpus should like this:

In [76]:
print(words[-101:])

['Sedan', 'riktade', 'han', 'också', 'ett', 'par', 'ord', 'till', 'Klara', 'Gulla', 'Hon', 'hade', 'fått', 'större', 'kärlek', 'av', 'sina', 'föräldrar', 'än', 'någon', 'annan', 'han', 'visste', 'och', 'sådan', 'kärlek', 'måste', 'vändas', 'i', 'välsignelse', 'Då', 'prästen', 'sade', 'detta', 'kom', 'alla', 'människor', 'att', 'se', 'bort', 'mot', 'Klara', 'Gulla', 'och', 'de', 'förundrade', 'sig', 'över', 'vad', 'de', 'såg', 'Prästens', 'ord', 'tycktes', 'redan', 'ha', 'gått', 'i', 'uppfyllelse', 'Där', 'stod', 'Klara', 'Fina', 'Gulleborg', 'ifrån', 'Skrolycka', 'hon', 'som', 'var', 'uppkallad', 'efter', 'själva', 'solen', 'vid', 'sina', 'föräldrars', 'grav', 'och', 'lyste', 'som', 'en', 'förklarad', 'Hon', 'var', 'likaså', 'vacker', 'som', 'den', 'söndagen', 'då', 'hon', 'gick', 'till', 'kyrkan', 'i', 'den', 'röda', 'klänningen', 'om', 'inte', 'vackrare']


### Counting unigrams and bigrams

Read and try programs to compute the frequency of unigrams and bigrams of the training set: [<a
            href="https://github.com/pnugues/ilppp/tree/master/programs/ch05/python">Program folder</a>].

#### Unigrams

In [77]:
def unigrams(words):
    frequency = {}
    for i in range(len(words)):
        if words[i] in frequency:
            frequency[words[i]] += 1
        else:
            frequency[words[i]] = 1
    return frequency
fq = unigrams(separated)
list(fq.items())[:20]

[('<s>', 58469),
 ('selma', 52),
 ('lagerlöf', 269),
 ('nils', 87),
 ('holgerssons', 6),
 ('underbara', 23),
 ('resa', 317),
 ('genom', 688),
 ('sverige', 56),
 ('</s>', 58470),
 ('första', 525),
 ('bandet', 6),
 ('bokutgåva', 11),
 ('albert', 15),
 ('bonniers', 11),
 ('förlag', 11),
 ('stockholm', 77),
 ('1907', 2),
 ('den', 11621),
 ('kristliga', 2)]

We compute the frequencies.

In [78]:
frequency = unigrams(words)
list(frequency.items())[:20]

[('Selma', 50),
 ('Lagerlöf', 268),
 ('Nils', 86),
 ('Holgerssons', 5),
 ('underbara', 22),
 ('resa', 313),
 ('genom', 678),
 ('Sverige', 55),
 ('Första', 8),
 ('bandet', 6),
 ('Bokutgåva', 11),
 ('Albert', 14),
 ('Bonniers', 11),
 ('förlag', 11),
 ('Stockholm', 76),
 ('DEN', 24),
 ('KRISTLIGA', 1),
 ('DAGVISAN', 1),
 ('Den', 1370),
 ('signade', 3)]

#### Bigrams

In [79]:
def bigrams(words):
    bigrams = []
    for i in range(len(words) - 1):
        bigrams.append((words[i], words[i + 1]))
    frequency_bigrams = {}
    for i in range(len(words) - 1):
        if bigrams[i] in frequency_bigrams:
            frequency_bigrams[bigrams[i]] += 1
        else:
            frequency_bigrams[bigrams[i]] = 1
    return frequency_bigrams
fq_bi = bigrams(separated)
list(fq_bi.items())[:20]

[(('<s>', 'selma'), 6),
 (('selma', 'lagerlöf'), 10),
 (('lagerlöf', 'nils'), 1),
 (('nils', 'holgerssons'), 6),
 (('holgerssons', 'underbara'), 4),
 (('underbara', 'resa'), 4),
 (('resa', 'genom'), 6),
 (('genom', 'sverige'), 5),
 (('sverige', '</s>'), 19),
 (('</s>', '<s>'), 56713),
 (('<s>', 'första'), 11),
 (('första', 'bandet'), 1),
 (('bandet', 'bokutgåva'), 2),
 (('bokutgåva', 'albert'), 11),
 (('albert', 'bonniers'), 11),
 (('bonniers', 'förlag'), 11),
 (('förlag', 'stockholm'), 10),
 (('stockholm', '1907'), 2),
 (('1907', '</s>'), 2),
 (('<s>', 'den'), 1355)]

In [80]:
frequency_bigrams = bigrams(words)
list(frequency_bigrams.items())[:20]

[(('Selma', 'Lagerlöf'), 9),
 (('Lagerlöf', 'Nils'), 1),
 (('Nils', 'Holgerssons'), 5),
 (('Holgerssons', 'underbara'), 3),
 (('underbara', 'resa'), 3),
 (('resa', 'genom'), 5),
 (('genom', 'Sverige'), 4),
 (('Sverige', 'Första'), 1),
 (('Första', 'bandet'), 1),
 (('bandet', 'Bokutgåva'), 2),
 (('Bokutgåva', 'Albert'), 11),
 (('Albert', 'Bonniers'), 11),
 (('Bonniers', 'förlag'), 11),
 (('förlag', 'Stockholm'), 11),
 (('Stockholm', 'DEN'), 1),
 (('DEN', 'KRISTLIGA'), 1),
 (('KRISTLIGA', 'DAGVISAN'), 1),
 (('DAGVISAN', 'Den'), 1),
 (('Den', 'signade'), 2),
 (('signade', 'dag'), 2)]

In the report, tell what is the possible number of bigrams and their real number? Explain why such a difference. What would be the possible number of 4-grams.

Propose a solution to cope with bigrams unseen in the corpus.

### Computing the likelihood of a sentence

#### Unigrams

Write a program to compute a sentence's probability using unigrams. You may find useful the dictionaries that we saw in the mutual information program: [<a href="https://github.com/pnugues/ilppp/tree/master/programs/ch05/python">Program folder</a>]. Your function will return the perplexity.

Your function should print and tabulate the results as in the examples below with the sentence _Det var en gång en katt som hette Nils_.

```
=====================================================
wi 	 C(wi) 	 #words 	 P(wi)
=====================================================
det 	 21108 	 1041631 	 0.0202643738521607
var 	 12090 	 1041631 	 0.01160679741674355
en 	 13514 	 1041631 	 0.01297388422579589
gång 	 1332 	 1041631 	 0.001278763784871994
en 	 13514 	 1041631 	 0.01297388422579589
katt 	 16 	 1041631 	 1.5360525944408337e-05
som 	 16288 	 1041631 	 0.015637015411407686
hette 	 97 	 1041631 	 9.312318853797554e-05
nils 	 87 	 1041631 	 8.352285982272032e-05
</s> 	 59047 	 1041631 	 0.056687060964967444
=====================================================
Prob. unigrams:	 5.361459667285409e-27
Geometric mean prob.: 0.0023600885848765307
Entropy rate:	 8.726943273141258
Perplexity:	 423.71290908655254
```

In [81]:
def unigram_lm(f, text):
    n = len(text)
    c_wi = {}
    P_wi = {}
    wc = 0
    for key in f.keys():
        wc += f[key]
    for word in text:
        c = f[word]
        c_wi[word] = c
        P_wi[word] = c / wc
    print("=====================================================\n")
    print("wc      C(wi)      #words      P(wi)\n")
    print("=====================================================\n")
    Prob_Unigrams = 1
    for word in text:
        print(word + "      " + str(c_wi[word]) + "      " + str(wc) + "      " + str(P_wi[word]) + "\n")
        Prob_Unigrams = Prob_Unigrams * P_wi[word]
    print("=====================================================\n")
    print("Prob. Unigrams: " + str(Prob_Unigrams))
    Geo_mean_p = math.pow(Prob_Unigrams, 1/n)
    print("Geometric mean Prob.: " + str(Geo_mean_p))
    
    H_L = -(1/n)*math.log2(Prob_Unigrams)
    print("Entropy Rate: " + str(H_L))
    print("Perplexity: " + str(1/Geo_mean_p))
    return 1/Geo_mean_p
    


In [82]:
sentence = 'det var en gång en katt som hette nils </s>'
sent_words = sentence.split()
sent_words

['det', 'var', 'en', 'gång', 'en', 'katt', 'som', 'hette', 'nils', '</s>']

In [83]:
#perplexity_unigrams = unigram_lm(frequency, sent_words)
perplexity_unigrams = unigram_lm(unigrams(separated), sent_words)


wc      C(wi)      #words      P(wi)


det      21104      1040879      0.020275171273510178

var      12090      1040879      0.011615182936729437

en      13510      1040879      0.012979414514078967

gång      1328      1040879      0.001275844742760686

en      13510      1040879      0.012979414514078967

katt      16      1040879      1.5371623406755253e-05

som      16288      1040879      0.015648312628076845

hette      97      1040879      9.319046690345372e-05

nils      87      1040879      8.358320227423168e-05

</s>      58470      1040879      0.05617367628706122


Prob. Unigrams: 5.327325216987506e-27
Geometric mean Prob.: 0.0023585816805888477
Entropy Rate: 8.727864720623083
Perplexity: 423.9836204232445


In [84]:
perplexity_unigrams = int(perplexity_unigrams)
perplexity_unigrams

423

#### Bigrams

Write a program to compute the sentence probability using bigrams. Your function will tabulate and print the results as below. It will return the perplexity.

```
=====================================================
wi 	 wi+1 	 Ci,i+1 	 C(i) 	 P(wi+1|wi)
=====================================================
<s>	 det 	 5672 	 59047 	 0.09605907158704083
det 	 var 	 3839 	 21108 	 0.1818741709304529
var 	 en 	 712 	 12090 	 0.058891645988420185
en 	 gång 	 706 	 13514 	 0.052242119283705785
gång 	 en 	 20 	 1332 	 0.015015015015015015
en 	 katt 	 6 	 13514 	 0.0004439840165754033
katt 	 som 	 2 	 16 	 0.125
som 	 hette 	 45 	 16288 	 0.002762770137524558
hette 	 nils 	 0 	 97 	 0.0 	 *backoff: 	 8.352285982272032e-05
nils 	 </s> 	 2 	 87 	 0.022988505747126436
=====================================================
Prob. bigrams:	 2.376007803503683e-19
Geometric mean prob.: 0.013727289294133601
Entropy rate:	 6.186809422848149
Perplexity:	 72.84759420254609
```

In [85]:
def bigram_lm(f, f_b, text): #TODO
    n = len(text)
    wc = 0
    for key in f.keys():
        wc += f[key]
    
    print("=====================================================\n")
    print("wi      wi+1      ci,i+1      C(i)      P(wi+1|wi)\n")
    print("=====================================================\n")
    word0 = "<s>"
    word1 = text[0]
    try:
        c_i_ip1 = f_b[(word0, word1)]
    except KeyError:
        c_i_ip1 = 0
    try:    
        c_i = f[word0]
    except KeyError:
        c_i = 0
    P_wip1_wi = c_i_ip1/c_i
    if (P_wip1_wi != 0.0):
            print(f"{word0}     {word1}       {c_i_ip1}       {c_i}        {P_wip1_wi} ")
    else:
            Prob_Bigrams = Prob_Bigrams * f[word0]/wc
            print(f"{word0}     {word1}       {c_i_ip1}       {c_i}        {P_wip1_wi} *backoff: {f[word0]/wc} ")
    Prob_Bigrams = f[word0]/wc
    for i in range(len(text)-1):
        word0 = text[i]
        word1 = text[i+1]
        try:
            c_i_ip1 = f_b[(word0, word1)]
        except KeyError:
            c_i_ip1 = 0
        try:    
            c_i = f[word0]
        except KeyError:
            c_i = 0
        P_wip1_wi = c_i_ip1/c_i
        if (P_wip1_wi != 0.0):
            print(f"{word0}     {word1}       {c_i_ip1}       {c_i}        {P_wip1_wi} ")
            Prob_Bigrams = Prob_Bigrams * P_wip1_wi
        else:
            Prob_Bigrams = Prob_Bigrams * f[word0]/wc
            print(f"{word0}     {word1}       {c_i_ip1}       {c_i}        {P_wip1_wi} *backoff: {f[word0]/wc} ")
        
    #breakpoint()
    
    print("=====================================================\n")
    print("Prob. Bigrams: " + str(Prob_Bigrams))
    Geo_mean_p = math.pow(Prob_Bigrams, 1/n)
    print("Geometric mean Prob.: " + str(Geo_mean_p))
    
    H_L = -(1/n)*math.log2(Prob_Bigrams)
    print("Entropy Rate: " + str(H_L))
    print("Perplexity: " + str(1/Geo_mean_p))
    
    return 1/Geo_mean_p

#bigram_lm(unigrams(separated), bigrams(separated), sent_words)
#bigram_lm(frequency, frequency_bigrams, sent_words)
#frequency


In [86]:
#unigrams(separated)

In [87]:
#perplexity_bigrams = bigram_lm(frequency, frequency_bigrams, sent_words)
perplexity_bigrams = bigram_lm(unigrams(separated), bigrams(separated), sent_words)


wi      wi+1      ci,i+1      C(i)      P(wi+1|wi)


<s>     det       5565       58469        0.09517864167336537 
det     var       3838       21104        0.18186125852918877 
var     en       712       12090        0.058891645988420185 
en     gång       704       13510        0.052109548482605475 
gång     en       20       1328        0.015060240963855422 
en     katt       6       13510        0.00044411547002220575 
katt     som       2       16        0.125 
som     hette       45       16288        0.002762770137524558 
hette     nils       0       97        0.0 *backoff: 9.319046690345372e-05 
nils     </s>       2       87        0.022988505747126436 

Prob. Bigrams: 1.551320149485035e-19
Geometric mean Prob.: 0.013154372726397044
Entropy Rate: 6.248313735348671
Perplexity: 76.02034857909169


In [88]:
perplexity_bigrams = int(perplexity_bigrams)
perplexity_bigrams

76

In addition to this sentence, _Det var en gång en katt som hette Nils_, write five other sentences that will form your test set and run your programs on them. You will insert them in your report.

### Online prediction of words

You will now carry out an online prediction of words. You will consider two cases:
1. Prediction of the current word a user is typing;
2. Prediction of the next word.

Ideally, you would write a loop that reads the words and apply the models while typing. As the Jupyter labs are not designed for interactive input and output, we will simplify the experimental settings with constant strings at a given time of the input.  

We will assume the user is typing the phrase: _Det var en gång_. 

#### Trigrams

To have a more accurate prediction, you will use a trigram counting function. Program this function following the model of your bigram counting function.

In [89]:
def trigrams(text): #returns dictionary -> (key, value), key = (str, str, str) , value = ?def bigrams(words):
    trigrams = []
    for i in range(len(words) - 2):
        trigrams.append((words[i], words[i + 1], words[i + 2]))
    frequency_trigrams = {}
    for i in range(len(words) - 2):
        if trigrams[i] in frequency_trigrams:
            frequency_trigrams[trigrams[i]] += 1
        else:
            frequency_trigrams[trigrams[i]] = 1
    return frequency_trigrams
fq_tr = trigrams(separated)
list(fq_tr.items())[:20]

[(('Selma', 'Lagerlöf', 'Nils'), 1),
 (('Lagerlöf', 'Nils', 'Holgerssons'), 1),
 (('Nils', 'Holgerssons', 'underbara'), 3),
 (('Holgerssons', 'underbara', 'resa'), 3),
 (('underbara', 'resa', 'genom'), 3),
 (('resa', 'genom', 'Sverige'), 3),
 (('genom', 'Sverige', 'Första'), 1),
 (('Sverige', 'Första', 'bandet'), 1),
 (('Första', 'bandet', 'Bokutgåva'), 1),
 (('bandet', 'Bokutgåva', 'Albert'), 2),
 (('Bokutgåva', 'Albert', 'Bonniers'), 11),
 (('Albert', 'Bonniers', 'förlag'), 11),
 (('Bonniers', 'förlag', 'Stockholm'), 11),
 (('förlag', 'Stockholm', 'DEN'), 1),
 (('Stockholm', 'DEN', 'KRISTLIGA'), 1),
 (('DEN', 'KRISTLIGA', 'DAGVISAN'), 1),
 (('KRISTLIGA', 'DAGVISAN', 'Den'), 1),
 (('DAGVISAN', 'Den', 'signade'), 1),
 (('Den', 'signade', 'dag'), 2),
 (('signade', 'dag', 'som'), 2)]

In [90]:
frequency_trigrams = trigrams(words)
frequency_trigrams[('det', 'var', 'en')]

86

#### Prediction

The user starts typing _Det var en gång_. After the 2nd character, your program tries to help the user with suggested words.

In [46]:
starting_text = 'De'.lower()
starting_text

'de'

Write a program to rank the five first candidates at this point. Assign these predictions in a list that you will call `current_word_predictions_1`. Note that you are starting a sentence and you can then use the bigram frequencies. Write a sorting key that will enable you have a deterministic ranking of the words or bigrams with identical frequencies: When two words have the same frequency, you will sort them by alphabetic order. You can do this with a tuple.

In [92]:
cand_nbr = 5

In [107]:
def predict1w(st_text):

    big = bigrams(separated) 
    suggestions = set([word for word in big if word[0].startswith(st_text)])
    
    tmp = sorted(suggestions)
    
    return tmp

current_word_predictions_1 = predict1w(starting_text)
current_word_predictions_1

[('de', '</s>'),
 ('de', 'absolut'),
 ('de', 'adopterade'),
 ('de', 'af'),
 ('de', 'affallne'),
 ('de', 'aflägsnare'),
 ('de', 'akta'),
 ('de', 'aldrig'),
 ('de', 'alla'),
 ('de', 'alldeles'),
 ('de', 'allesamman.'),
 ('de', 'allesammans'),
 ('de', 'allihop'),
 ('de', 'allra'),
 ('de', 'alls'),
 ('de', 'allt'),
 ('de', 'allthittills'),
 ('de', 'alltid'),
 ('de', 'alltjämt'),
 ('de', 'alltmer'),
 ('de', 'alltså'),
 ('de', 'alomkransade'),
 ('de', 'amerikanska'),
 ('de', 'anade'),
 ('de', 'andan'),
 ('de', 'andliga'),
 ('de', 'andra'),
 ('de', 'andra.'),
 ('de', 'andras'),
 ('de', 'anfallit'),
 ('de', 'anklagas'),
 ('de', 'ankommande'),
 ('de', 'anlände'),
 ('de', 'annars'),
 ('de', 'annat'),
 ('de', 'anordna'),
 ('de', 'anse'),
 ('de', 'anser'),
 ('de', 'ansträngda'),
 ('de', 'ansträngde'),
 ('de', 'anställde'),
 ('de', 'ansågo'),
 ('de', 'antastades'),
 ('de', 'antogo'),
 ('de', 'anträdde'),
 ('de', 'anvisade'),
 ('de', 'apotekares'),
 ('de', 'arbeta'),
 ('de', 'arbetade'),
 ('de', 'ar

In [ ]:
current_word_predictions_1

Let us now suppose that the user has typed: _Det var en_. After detecting a space, your program starts predicting a next possible word.

In [ ]:
current_text = "Det var en ".lower()
current_text

Tokenize this text and return a list of tokens. Call it `tokens`.

In [ ]:
# Write your code here

In [ ]:
tokens

Write a program to propose the five next possible words ranked by frequency using a trigram model. Assign these predictions to a variable that you will call `next_word_predictions`. Write a sorting key that will enable you have a deterministic ranking of the words or bigrams with identical frequencies: When two words have the same frequency, you will sort them by alphabetic order. You can do this with a tuple.

In [ ]:
# Write your code here

In [ ]:
next_word_predictions

Finally, let us suppose that the user has typed _Det var en g_, rank the five possible candidates. Assign these predictions in a list that you will call `current_word_predictions_2`

In [ ]:
current_text = "Det var en g".lower()

In [ ]:
# Write your code here

In [ ]:
current_word_predictions_2

## Checked answers

The automatic marking system will check these answers: `(perplexity_unigrams, perplexity_bigrams, current_word_predictions_1, next_word_predictions, current_word_predictions_2)`

In [ ]:
(perplexity_unigrams, perplexity_bigrams, current_word_predictions_1, next_word_predictions, current_word_predictions_2)

## Submission

When you have written all the code and run all the cells, fill in your ID and as well as the name of the notebook.

In [ ]:
STIL_ID = ["student1", "student2"] # Write your stil ids as a list
CURRENT_NOTEBOOK_PATH = os.path.join(os.getcwd(), 
                                     "2-language_models_solution.ipynb") # Write the name of your notebook

The submission code will send your answer. It consists of the perplexities and predictions.

In [ ]:
import json
ANSWER = json.dumps({'perplexity_unigrams': perplexity_unigrams,
                     'perplexity_bigrams': perplexity_bigrams,
                     'current_word_predictions_1': current_word_predictions_1,
                     'next_word_predictions': next_word_predictions,
                     'current_word_predictions_2': current_word_predictions_2})
ANSWER

The checking script will accept perplexities with a margin, like for instance:
```
(421, 72, ['det', 'de', 'den', 'detta', 'denna'], ['stor', 'liten', 'gammal', 'god', 'sådan'], ['gammal', 'god', 'gång', 'ganska', 'glädje'])
```

Now the moment of truth:
1. Save your notebook and
2. Run the cells below

In [ ]:
SUBMISSION_NOTEBOOK_PATH = CURRENT_NOTEBOOK_PATH + ".submission.bz2"

In [ ]:
ASSIGNMENT = 2
API_KEY = "f581ba347babfea0b8f2c74a3a6776a7"

# Copy and compress current notebook
with bz2.open(SUBMISSION_NOTEBOOK_PATH, mode="wb") as fout:
    with open(CURRENT_NOTEBOOK_PATH, "rb") as fin:
        fout.write(fin.read())

In [ ]:
res = requests.post("https://vilde.cs.lth.se/edan20checker/submit", 
                    files={"notebook_file": open(SUBMISSION_NOTEBOOK_PATH, "rb")}, 
                    data={
                        "stil_id": STIL_ID,
                        "assignment": ASSIGNMENT,
                        "answer": ANSWER,
                        "api_key": API_KEY,
                    },
                   verify=True)

# from IPython.display import display, JSON
res.json()

## Turning in your assignment

Now your are done with the program. To complete this assignment, you will:
1. Write a short individual report on your program. Do not forget to include the unigram and bigram tables for five sentences and _Det var en gång en katt som hette Nils_.
2. Execute the Jupyter notebook by Peter Norvig here: <a href="http://nbviewer.jupyter.org/url/norvig.com/ipython/How%20to%20Do%20Things%20with%20Words.ipynb">https://nbviewer.jupyter.org/url/norvig.com/ipython/How to Do Things with Words.ipynb</a>. Just run all the cells and be sure that you understand the code. You will find the data here: <a href="http://norvig.com/ngrams/">http://norvig.com/ngrams/</a>.
3. In your report, after the description of your program, you will describe one experiment with Norvig's notebook and a long string of words your will create yourself or copy from a text you like. You will remove all the punctuation and white spaces from this string. You will set this string in lowercase letters. You will just add a cell at the end of Sect. 7 in Norvig's notebook, where you will use your string and run the notebook cell with the <tt>segment()</tt> and <tt>segment2()</tt> functions. You will comment the segmentation results you obtain with the unigram and bigram models.

Submit your report as well as your notebook (for archiving purposes) to Canvas: https://canvas.education.lu.se/. To write your report, you can either
1. Write directly your text in Canvas, or
2. Use Latex and Overleaf (www.overleaf.com). This will probably help you structure your text. You will then upload a PDF file in Canvas.

The submission deadline is September 24, 2021.